# Vamos aprender a trabalhar com PDF usando o Python

- Regra geral: PDF foi feito justamente para bloquear muita coisa, então não é fácil "brincar" com um pdf
- Mesmo assim, Python tem várias bibliotecas que vão nos ajudar, vamos focar em 2:
    - PyPDF2
    - Tabula
- Ler e extrair informações de um PDF a gente consegue fazer.
- Escrever e Editar, aí já é outra história

### Para os nossos exemplos, vamos avaliar o Release de Resultados do 3º e 4º Trimestre de 2020 da Magazine Luiza

#### 1º Objetivo: Queremos conseguir separar apenas o DRE do Release de Resultados (Página 14) para enviar para a Diretoria, como fazemos?
    - Separar as páginas de um pdf

In [9]:
import PyPDF2 as pyf
from pathlib import Path

pyf.PdfFileReader # para ler um arquivo em PDF
pyf.PdfFileWriter # criar/escrever um arquivo em PDF
pyf.PdfFileMerger # mesclar arquivos PDF

nomeArquivoPdf='MGLU_ER_3T20_POR.pdf'
arquivoPdf=pyf.PdfReader(nomeArquivoPdf)
display(arquivoPdf)

numPagina=1
# torna o arquivo iteravel, o for vai rolar pagina por pagina
for pagina in arquivoPdf.pages: 
    # cria o arquivo com uma pagina apenas
    arquivoNovo=pyf.PdfWriter()
    arquivoNovo.add_page(pagina)
    # salva o arquivo
    # abre o arquivo Arquivo1.pdf em paginas\\ no mode='wb' (modo writable)
    # escreve/salva o arquivo novo no arquivo final
    with Path(f'paginas\\Arquivo{numPagina}.pdf').open(mode='wb') as arquivoFinal:
        arquivoNovo.write(arquivoFinal)
        numPagina+=1

#### 2º Objetivo: Com o Release de Resultados já separado página por página, queremos incluir apenas as Páginas de Destaque (Página 1), DRE (Página 14) e Balanço (Página 16).
    - Juntar vários pdfs em 1

In [10]:
paginasSolicitadas=[1,14,16]

arquivoNovo=pyf.PdfWriter()
for numPagina in paginasSolicitadas:
    # Pegar o arquivo numero da pagina
    # Adicionar no novo PDF
    nomeArquivo=f'paginas\\Arquivo{numPagina}.pdf'
    arquivoPdf=pyf.PdfReader(nomeArquivo)
    pagina=arquivoPdf.pages[0] # tem que ter o indice zero porque por natureza é uma lista de paginas, na ausencia desse passara a lista inteira (mesmo que so haja um elemento)
    arquivoNovo.add_page(pagina)

# salvar o novo PDF
with Path(f'Consolidado.pdf').open(mode='wb') as arquivoFinal:
    arquivoNovo.write(arquivoFinal)

### Extra: Para adicionar todas as páginas de 2 pdfs

In [11]:
pdfMesclado=pyf.PdfMerger()

arquivo1='MGLU_ER_3T20_POR.pdf'
arquivo2='MGLU_ER_4T20_POR.pdf'

# so funciona com um append de cada vez
pdfMesclado.append(arquivo1)
pdfMesclado.append(arquivo2)

with Path(f'ConsolidadoAppend.pdf').open(mode='wb') as arquivoFinal:
    pdfMesclado.write(arquivoFinal)

# Funcionalidades que podem ser úteis:

- Inserir arquivo no meio do outro
- Quero colocar dentro do Resultado do 4T20 os destaques do 3T20 para poder comparar os 2 dentro do mesmo relatório

In [12]:
pdfMesclado=pyf.PdfMerger()

pdfMesclado.append(arquivo1)

# adiciona o arquivo 2 APÓS a pagina número 1 do arquivo 1, nesse caso 1 seria efetivamente a primeira folha e nao 0 como de costume
pdfMesclado.merge(1,arquivo2)

with Path(f'ConsolidadoMerge.pdf').open(mode='wb') as arquivoFinal:
    pdfMesclado.write(arquivoFinal)

- Rodar Página

In [13]:
pdfRodar=pyf.PdfReader(arquivo1)

pdfFinal=pyf.PdfWriter()
for pagina in pdfRodar.pages:
    # 90 deita a pagina, 180 coloca ao contrario
    pagina.rotate(90)
    pdfFinal.add_page(pagina)

with Path(f'Rodado.pdf').open(mode='wb') as arquivoFinal:
    pdfFinal.write(arquivoFinal)

# Trabalhando com Textos e Informações Dentro do PDF

#### 1º Objetivo: Quero identificar como foram as Despesas com Vendas da MGLU
    - Pegar texto da página e identificar onde está essa informação

In [26]:
nomeArquivo='MGLU_ER_3T20_POR.pdf'
arquivo=pyf.PdfReader(nomeArquivo)

# imprime o numero de paginas do arquivo
numPaginas=len(arquivo.pages)
print(numPaginas)

# imprime as informacoes do arquivo
informacoesArquivo=arquivo.metadata
print(informacoesArquivo)


textoProcurado='| Despesas com Vendas'

# percorrendo todas as páginas
numPagina=1
for pagina in arquivo.pages:
    # pegar o que está escrito na página
    textoPagina=pagina.extract_text()
    # textoPagina=pagina.extract_text().replace('  ',' ').replace('\n \n ','&&&&&&').replace('\n','').replace('&&&&&&','\n') # tentativa de padronizar o texto

    # verificar se dentro do texto da pagina tem o texto procurado
    if textoProcurado in textoPagina:
        print(f'Está na página {numPagina}')
        textoFinal=textoPagina

    # se tiver, retornar qual o numero da pagina

    numPagina+=1

# print(textoFinal)

# descobre a posicao do primeiro caractere referente ao texto procurado
posicaoInicialTexto=textoFinal.find('| Despesas com Vendas')
print(posicaoInicialTexto)

# pesquisa a proxima barrinha do texto, o segundo argumento é a partir de que posicao ele vai procurar
posicaoFinalTexto=textoFinal.find('|', posicaoInicialTexto+1)
print(posicaoFinalTexto)

textoDespesa=textoFinal[posicaoInicialTexto:posicaoFinalTexto]
print(textoDespesa)

24
{'/Title': 'DESEMPENHO FINANCEIRO CONSOLIDADO', '/Author': 'an_rezende', '/Subject': 'Receita Bruta', '/Creator': 'Microsoft® Office Word 2007', '/CreationDate': "D:20201109183121-03'00'", '/ModDate': "D:20201109183121-03'00'", '/Producer': 'Microsoft® Office Word 2007'}
Está na página 10
Divulgação de Resultados  
3T20 
10 
 
  
 
| Despesas Operacionais  
 
R$ milhões  3T20  
Ajustado   % RL  3T19  
Ajustado   % RL   Var(%)  9M20  
Ajustado   % RL  9M19  
Ajustado   % RL   Var(%)  
  Despesas com Vendas   (1.432,6)  -17,2%  (890,0)  -18,3%  61,0%  (3.487,2)  -18,2%  (2.309,1)  -17,1%  51,0%  
  Despesas Gerais e Administrativas   (240,7)  -2,9%  (207,1)  -4,3%  16,2%  (617,3)  -3,2%  (498,2)  -3,7%  23,9%  
 Subtotal      (1.673,3)  -20,1%      (1.097,1)  -22,6%  52,5%      (4.104,5)  -21,5%      (2.807,4)  -20,8%  46,2%  
  Perdas em Liquidação Duvidosa   (25,4)  -0,3%  (20,2)  -0,4%  25,4%  (84,5)  -0,4%  (45,8)  -0,3%  84,3%  
  Outras Receitas Operacionais, Líquidas   15,2  0,

#### 2º Objetivo: Quero analisar o DRE (sem ajuste - Página 5)
    - Para ler tabelas em pdf, use o tabula (é ninja)

#### 3º Objetivo: Quero analisar o Capital de Giro e os Investimentos (ambas as tabelas na página 12)
    - Páginas com mais de 1 tabela

#### O que fazer quando o tabula não consegue ler alguma linha da tabela? Como o cabeçalho, no nosso caso?

# Outro método que pode ser útil algum dia: Captar Imagem em um pdf
    - biblioteca pikepdf

In [15]:
from pikepdf import Pdf, PdfImage

filename = "MGLU_ER_3T20_POR.pdf"
example = Pdf.open(filename)

n_arquivo = 0
for i, pagina in enumerate(example.pages):
    for nome, imagem in pagina.images.items():
        imagem_salvar = PdfImage(imagem)
        imagem_salvar.extract_to(fileprefix=f'imgs/Pagina_{i:02}_{n_arquivo}')
        n_arquivo += 1

ModuleNotFoundError: No module named 'pikepdf'

# Substituir texto no pdf tipo contrato

- Não recomendo fazer diretamente pelo Python. Realmente do que vi a melhor opção me parece o Word fazer isso
- Caso precise automatizar, automatize o processo fazendo ele pelo Word
- Quem quiser MUITO fazer isso pelo Python, tem um link aqui que vai te ajudar de uma solução que achei que funciona. Tem seus bugs/cuidados especiais, mas funciona: https://pdf.co/samples/pdf-co-web-api-replace-text-from-pdf-python-replace-text-from-url